# Preparation

### Imports

In [212]:
using BenchmarkTools

using NBInclude;
@nbinclude("Rosenbrock_BFGS.ipynb");
@nbinclude("Rosenbrock_GD_Newton.ipynb");

[19.91477155, 0.06956248, 0.0669018, 0.03908957, 0.01534347, 0.00551095, 0.00269375, 0.0004362, 3.463e-5, 3.3e-7]
[6.3274252, 0.22158139, 0.20125658, 0.1405162, 0.08145223, 0.04680074, 0.03311067, 0.01185387, 0.00446461, 0.00148593]
function value in iteration 1 for x=[13.501753999316922, -190.87813479251096] is 1.3926151364185793e7
function value in iteration 2 for x=[-2.0154209991730647e6, 74444.22103532378] is 1.6499208136288062e27
function value in iteration 3 for x=[3.274592940223273e21, 8.123843459671509e14] is 1.1498184808922469e88
function value in iteration 4 for x=[-1.4045330236543517e67, 2.1445917848320198e45] is 3.8915966377182745e270
function value in iteration 5 for x=[1.108296230043211e204, 3.945426029071271e136] is Inf
function value in iteration 5175 for x=[0.9968426609340175, 0.9936826322371464] is 9.98481353946191e-6
function value in iteration 1 for x=[0.17071072711366356, 0.02906039695181256] is 0.6877213665188894
function value in iteration 2 for x=[0.986658374759

In [196]:
function rosenbrock_performance(fun, samples, dimensions, max_iters, max_error=0)
    timed_history = []
    for j in 1:samples
        timed = @timed fun(rand(dimensions), max_iters, max_error)
        push!(timed_history, timed)
    end
    
    mean_iters = sum(map(h->h.value[3], timed_history))/samples
    mean_error = sum(map(h->last(h.value[2]), timed_history))/samples
    mean_exec_time = sum(map(h->h.time, timed_history))/samples
    mean_alloc_bytes = sum(map(h->h.bytes, timed_history))/samples
    
    return mean_iters, mean_error, mean_exec_time, mean_alloc_bytes
end

rosenbrock_performance (generic function with 2 methods)

### Adapters

In [213]:
gd(x, max_iters, max_error) = rosenbrock_gradient_descent(x, rate, max_iters, max_error)
lbfgs(x, max_iters, max_error) = rosenbrock_LBFGS(x, m, max_iters, max_error)

lbfgs (generic function with 1 method)

# Mean performance of 100 runs for 2 dimens

In [223]:
SAMPLES = 100;
DIMENS = 2;
ITERS = 10000;
ERROR = 10e-6;

rate = 0.001953125; #GradientDescent
m = 4; #L-BFGS

### Gradient Descent performance

In [215]:
rosenbrock_performance(gd, SAMPLES, DIMENS, ITERS, ERROR)

(5373.44, 9.991714084738992e-6, 0.006011725880000001, 4.66000704e6)

### Newton's Method performance

In [216]:
rosenbrock_performance(rosenbrock_newton, SAMPLES, DIMENS, ITERS, ERROR)

(4.18, 1.1906558255356876e-6, 0.00026203402, 255100.96)

### BFGS performance

In [217]:
rosenbrock_performance(rosenbrock_BFGS, SAMPLES, DIMENS, ITERS, ERROR)

(10.11, 2.4392e-6, 0.00093020877, 495672.58)

### L-BFGS performance

In [224]:
rosenbrock_performance(lbfgs, SAMPLES, DIMENS, ITERS, ERROR)

(10.51, 2.5800999999999997e-6, 0.0006146762, 490205.44)

# Mean performance of 100 runs for 4 dimens

In [225]:
DIMENS = 4;

### Gradient Descent performance

In [226]:
rosenbrock_performance(gd, SAMPLES, DIMENS, ITERS, ERROR)

(10000.0, 1.7682356558224654, 0.01386293305, 1.1592096e7)

### Newton's Method performance

In [227]:
rosenbrock_performance(rosenbrock_newton, SAMPLES, DIMENS, ITERS, ERROR)

(1710.08, 0.6295164370112346, 0.00830089069, 1.213725888e7)

### BFGS performance

In [228]:
rosenbrock_performance(rosenbrock_BFGS, SAMPLES, DIMENS, ITERS, ERROR)

(20.37, 2.5608e-6, 0.00070068644, 908448.48)

### L-BFGS performance

In [229]:
rosenbrock_performance(lbfgs, SAMPLES, DIMENS, ITERS, ERROR)

(23.85, 2.7256000000000002e-6, 0.00110630277, 1.06946208e6)

# Visualization of Rosenbrock value change for 2 dimens

# Visualization of Rosenbrock value change for 4 dimens